In [43]:
import requests
import json
import wikipedia as wk
import csv

In [47]:
def get_revisions_by_pageid(pageid, title=None, coordinates=None):
    headers = {'content-type': 'text/mediawiki-api-prettyprint-wrapped; charset=utf-8'}
    url = 'https://en.wikipedia.org/w/api.php'
    params = {
        'action': 'query', 
        'format': 'json', 
        'prop': 'revisions', 
        'pageids': pageid,
        'rvprop': 'timestamp|ids|comment',
        'rvlimit': 5
    }

    req = requests.post(url, params=params, headers=headers)

    revs = req.json()['query']['pages'][str(pageid)]['revisions']

    for rev in revs:
        rev['title'] = title
        rev['pageid'] = pageid
        rev['coordinates'] = coordinates

    return revs

def to_csv(_json):
    f = csv.writer(open("test.csv", "wb+"))

    # Write CSV Header, If you dont need that, remove this line
    _headers = ["pageid", "title", "coordinates", "timestamp", "revid", "comment"]
    f.writerow(_headers)

    i = 0
    for x in _json:
        row = [x[h] for h in _headers]
        
        if i % 100 == 0:
            print("Progress: %s" % i)
        f.writerow(row)
        i += 1

#action=query&format=json&prop=revisions&pageids&rvprop=timestamp%7Cuser%7Ccomment%7Ccontent

In [48]:
j = get_revisions_by_pageid(1000)
to_csv(j)

Progress: 0


[1000, None, None, u'2016-02-26T05:39:04Z', 706953291, u'']
[1000, None, None, u'2016-02-13T03:01:12Z', 704709963, u'/* David Suchet */']
[1000, None, None, u'2016-02-13T02:58:14Z', 704709611, u'/* Policeman */ Fixed typo']
[1000, None, None, u'2016-02-13T02:54:22Z', 704709152, u'/* Policeman */']
[1000, None, None, u'2016-02-13T02:51:48Z', 704708881, u'/* Policeman */ Corrected style']
